<a href="https://colab.research.google.com/github/ehBeak/animationProject/blob/main/BackgroundMatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import os

모델 다운

In [ ]:
!gdown https://drive.google.com/uc?id=1ErIAsB_miVhYL9GDlYUmfbqlV293mSYf -O model.pth -q

모델 실행 코드 클론

In [ ]:
!git clone -q https://github.com/PeterL1n/BackgroundMattingV2


content 이동

In [ ]:
%cd /content
os.getcwd()

/content


'/content'

input 비디오의 첫 프레임 잘라서 content에 test.png로 저장

In [ ]:
vidcap = cv.VideoCapture('src.mp4')

if(vidcap.isOpened()): 
  ret, image = vidcap.read()
  cv.imwrite("./test.png",image)


BackgroundMattingV2 파일로 이동

In [ ]:
os.getcwd()
%cd BackgroundMattingV2

/content/BackgroundMattingV2


학습된 모델 실행, 결과값은 ouput에

In [ ]:
!python inference_video.py \
        --model-type mattingrefine \
        --model-backbone resnet50 \
        --model-backbone-scale 0.25 \
        --model-refine-mode sampling \
        --model-refine-sample-pixels 80000 \
        --model-checkpoint "/content/model.pth" \
        --video-src "/content/src.mp4" \
        --video-bgr "/content/test.png" \
        --output-dir "/content/output/" \
        --output-type com fgr pha err ref

Directory /content/output/ already exists. Override? [Y/N]: Y
  0% 0/274 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "inference_video.py", line 195, in <module>
    pha, fgr, _, _, err, ref = model(src, bgr)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/drive/My Drive/test/BackgroundMattingV2/model/model.py", line 162, in forward
    assert src.size() == bgr.size(), 'src and bgr must have the same shape'
AssertionError: src and bgr must have the same shape


ouput으로 이동

In [ ]:
os.getcwd()
%cd /content/output

/content/output


초록 부분 배경 제거

In [ ]:
# read the image
image_bgr = cv.imread('image_bgr.png')
# get the image dimensions (height, width and channels)
h, w, c = image_bgr.shape
# append Alpha channel -- required for BGRA (Blue, Green, Red, Alpha)
image_bgra = np.concatenate([image_bgr, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
# create a mask where white pixels ([255, 255, 255]) are True
green = np.all(image_bgr == [120, 255, 155], axis=-1)
# change the values of Alpha to 0 for all the white pixels
image_bgra[green, -1] = 0
# save the image
cv2.imwrite('image_bgra.png', image_bgra)

프레임 단위로 잘라서 저장

In [ ]:
vidcap = cv.VideoCapture('com.mp4')
success,image_bgr = vidcap.read()

count = 1
success = True

while success:
  success,image_bgr = vidcap.read()
  h, w, c = image_bgr.shape
  image_bgra = np.concatenate([image_bgr, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
  image_bgra[green, -1] = 0
  green = np.all(image_bgr == [155, 255, 120], axis=-1)
  cv.imwrite("/Your/img/save/path/%d.jpg" % count, image)
  print("saved image %d.jpg" % count)
  
  if cv.waitKey(10) == 27:                    
      break
  count += 1